# Artificial Neural Networks

Artifical neural networks (ANNs) are applied as a supervised learning technique to classify data into known classes.

![SegmentLocal](Images/curve_fit.png "curvefit")


\begin{align}
y & = W_1\,x4 + W_2\,x^3 + W_3\,x^2 + W_4\,x + W_5\\
\end{align}

You can think of them as a cousin of familiar regression or curve-fitting, but taken to higher dimensions. For example, when fitting a polynomial curve to a dataset we learn the values of the coefficients $W$ so that the model curve can accurately predict an outcome given input data. Similarly, for an ANN classifier, we learn a set of weights (coefficients) between the neurons so the the network can accurately predict a label for input data.

![SegmentLocal](Images/ANN_1layer.svg "ANN1")

In both cases the fitting procedure employs a measure of how well model performs. For curve fitting the distance between the data and model is minimised (e.g., $\chi^2$ or least-squared distance), while for an ANN classifier a *loss function* is minimised.

When training the classifier, the loss function desires the following state:

<img src="Images/loss_margin.jpg">

Scores inside the red region (or higher) will accumulate a loss, while scores below will accumulate zero loss.

## Stochastic Gradient Descent and mini-batches

Minimisation is done iteratively vis gradient descent:
 * Randomly initialise the weights between neurons
 * assess the performance of the network using the loss function
 * examine the slope of the loss surface and jump 'down-hill'
 * rinse and repeat until no improvement is seen

![SegmentLocal](Images/grad_descent.gif "descent")

This is superficially similar fitting a polynomial curve, however, there are two major differences:
 1. the weight coeefficients are not independent - the output of neurons are affected by all in the chain before them
 1. the very large datasets required by ANNs will not fit in memory
 
The first probem is addressed by back-probagating the gradients using the chain-rule. The second problem is solved by only training on a subset of the data, called a 'mini-batch' (or just 'batch'), and randomly changing the selection of this data every cycle.

## Implementing ANNs using Keras

In the last 10 years there has been an explosion in the availability of software for [deep learning](https://en.wikipedia.org/wiki/Comparison_of_deep-learning_software). One of the most popular is [Keras](https://keras.io/), which is a layer on top of Google's [TensorFlow](https://www.tensorflow.org/versions) machine learning and matrix manipulation library. (Alternatives are PyTorch, developed by Facebook, and CNTK by Microsoft). 

Performing the gradient descent step on even moderately large ANNs requires very intensive calculations. TensorFlow (and Keras) can take advantage of the parallel processing power offered by Graphics Processing Units (GPUs) to vastly accelerate the training step.


## Classifying handwriting - the MNIST dataset

Historically, one of the simplest tasks ANNs were applied to was handwriting recognition. In this notebook we will build an ANN to classify small images of handwritten digits [0 - 9]. This is a classic entry-level problem in machine learning.

The Modified National Institute of Standards and Technology (MNIST) dataset contains 60,000 training images and 10,000 testing images of handwritten digits as 28x28 pixel black & white images.

![SegmentLocal](Images/MNIST_grid.png "MNIST_grid")

These images are encoded with intensity values [0 - 255], so we usually scale them [0 - 1] by dividing by 255.0.

### Coding the classifier

Here we build a simple neural network classifier with two 'hidden layers'. The number of neurons in the *input layer* corresponds to each pixel of an image fed into the network. The number of neurons in the *output layer* corresponds to the possible classes.

![SegmentLocal](Images/ANN_2layers.png "ANN2")

For convenience the MNIST data is stored on disk as a Numpy save file containing a 3D array (images) or 1D array (labels 0 - 9). It is small enough to fit into memory. In this case, the data is already split into training ans tets sets.

In [ ]:
import numpy as np

# Load the training images and labels from Numpy save files
trainX = np.load('DATA/minimnist/train_images.npy')
trainY = np.load('DATA/minimnist/train_labels.npy')
print("Training array shape:", trainX.shape)
print("Labels shape", trainY.shape)

In [ ]:
# Load the testing images and labels from Numpy save files

Let's plot an individual image and display some statistics of the inetensity:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Show a randomly chosen image

Both the training and test data should be converted into floating point precision and scaled into the range [0 - 1]. Machine learning algorithms tend to expect data in this format, unless otherwise specified.

In [ ]:
# Scale the intensities [0, 1]

Just like with Scikit Learn, we need to re-encode our labels as one-hot vectors:

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# Create one-hot label vectors

We also need to flatten each image into a vector of 784 pixels so that it can be injected into the bottom layer of the network.

In [ ]:
# Reshape the train and test image stacks

Now it is time to define the architecture of our network. Keras supports the creation of complicated networks with multiple input or output branchs. However, we will define a simple straight-through network with two hidden layers containing 256 and 128 neurons, respectively. There should be 784 input neurons corresponding to the number of pixels in an image (28 * 28) and 10 output neurons, one for each digit.

In Keras, the 'Dense' layer represents a layer of neurons where each neuron will be connected to all other neurons in layers immediately above and below. 

The activation function scales the output of the neuron in a non-linear way, allowing the network to model non-linear systems.

![SegmentLocal](Images/SigmoidFunction.png "sigmoid")

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense

# Define 784-256-128-10 architecture using Keras

Once the model has been defined we need to pick a loss function to assess it's performance and run the gradient descent algorithm. There are also a number of tweaked optimiser algorithms avaialable, however SGD works fine as a default. 

The optimiser also accepts a 'learning rate', a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. Choosing the learning rate is challenging as a value too small may result in a long training process that could get stuck, whereas a value too large may result in learning a sub-optimal set of weights too fast or an unstable training process.

At this point we wlso need to choose a batch size (how much data can we fit in memory?) and number of ephocs to train for (how many times do we push a batch throughthe network to train it.

In [ ]:
from keras.optimizers import SGD

# Typical learning rate for SGD
learnRate = 0.01
opt = SGD(learnRate)

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# The run the stochastic gradient descent algorithm
H = model.fit(trainX, trainY, validation_data=(testX, testY),
              epochs=100, batch_size=64)

That's it! Training has finished and our model is ready to make prediction about the testing data. Let's just make a prediction about one image to illustrate.

In [ ]:
# Select a single image - a batch of 1

Scikit learn comes with a handy function to report useful classification statistics:

In [ ]:
from sklearn.metrics import classification_report

# Make predictions
predictions = model.predict(testX, batch_size=128)

# Print a formatted report
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=[str(x) for x in lb.classes_]))

We achieve an average precision of 89 %, which is not bad compared to our random chance of 1/10.

Let's go back and plot some key curves for the training process - loss and accuracy. These were stored in our history object.

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()